In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from multiprocessing import Pool
import pickle

In [2]:
def get_vacs(fpth):
    vacs = pd.read_excel(fpth)
    vacs = vacs.ids.astype('str').unique().tolist()
    vacs = [x for x in vacs if x.isdigit()]
    return vacs

In [3]:
def links_maker(vacs):
    links = []
    links_dict = {}
    for v_id in vacs:
        
        url = 'https://career18.sapsf.com/career?'
        url += 'career_ns=job_listing&company=paosberbanP2&navBarLevel=JOB_SEARCH&rcm_site_locale=ru_RU&'
        url += 'career_job_req_id={}&selected_lang=ru_RU&jobAlertController_jobAlertId'.format(v_id)
        url += '=&jobAlertCon=troller_jobAlertName=&_s.crb=ihl8rnb5dyInCnbwshM%2b3Ks%2b%2bCg%3d'
        links.append(url)
        links_dict[v_id] = url
    
    return links, links_dict

In [4]:
def get_info(url):
    res = requests.get(url)
    page = res.content
    soup = BeautifulSoup(page,'html.parser')
    result = soup.find_all(['h1','b','li','ul'])
    vac_description = []
    for el in result:
        vac_description.append(el.text)
    return vac_description


In [6]:
!ls

SF_Parser-Pool.ipynb          sf_parser_soup
SF_Parser-Pool_&_Simple.ipynb sf_selenium.py
SF_Parser.ipynb


In [7]:
fpth = 'sf_parser_soup/vac_id.xlsx'
vacs = get_vacs(fpth)

In [8]:
sf_links, sf_links_dict = links_maker(vacs)

In [9]:
data = []
for url in tqdm_notebook(sf_links[:30]):
    data.append(get_info(url))

In [10]:
with Pool(10) as p:
    records = p.map(get_info, sf_links[:30])

In [21]:
with open('vacs_records.pkl', 'wb') as f:
    pickle.dump(records,f)

In [13]:
# data

In [16]:
records

[['Карьерные возможности: Старший специалист в отдел технических средств охраны (606381)',
  '606381',
  '06.09.2019',
  'Безопасность',
  'Псковская область',
  'Псков',
  'ПАО Сбербанк',
  '\nадминистрирование серверной архитектуры и баз данных ПАК Центра мониторинга;\nподдержание работоспособности ПАК Центра мониторинга, включая мониторинг и устранение физических неисправностей (поддержка АРМ операторов центра мониторинга Псков);\nлокализация и организация устранения технических инцидентов (в случае необходимости привлечения IT специалистов банка);\nудаленная приемка новых объектов на ЦМ Банка, формирование карточек объектов в ПО "Центр охраны" ПАК "Андромеда";\nтехнический мониторинг ТСО объектов ГОСБ;\nадминистрирование электронных систем безопасности на объектах ГОСБ (удаленное управление системами охранной сигнализации);\nудаленный съем видео.\n',
  'администрирование серверной архитектуры и баз данных ПАК Центра мониторинга;',
  'поддержание работоспособности ПАК Центра монитор